<a href="https://colab.research.google.com/github/RoronoaZoro-0-7/Text-Classification---Transformers/blob/main/Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !mkdir -p data/full_dataset/
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

--2025-07-07 13:44:37--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv’

goemotions_1.csv    100%[===================>]  13.52M  --.-KB/s    in 0.1s    

2025-07-07 13:44:37 (101 MB/s) - ‘data/full_dataset/goemotions_1.csv’ saved [14174600/14174600]

--2025-07-07 13:44:38--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting 

In [2]:
!pip install transformers datasets scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
!pip uninstall -y transformers
!pip install transformers --upgrade --no-cache-dir

Found existing installation: transformers 4.53.0
Uninstalling transformers-4.53.0:
  Successfully uninstalled transformers-4.53.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 86.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
df1 = pd.read_csv("data/full_dataset/goemotions_1.csv")
df2 = pd.read_csv("data/full_dataset/goemotions_2.csv")
df3 = pd.read_csv("data/full_dataset/goemotions_3.csv")

# Merge into one DataFrame
df = pd.concat([df1, df2, df3], ignore_index=True)

# View first few rows
df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


# Take only clear examples and the ones which have atleast one emotion

In [6]:
df = df[df['example_very_unclear'] == False]

In [7]:
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
    'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
    'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
    'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise'
]

In [8]:
df['emotions'] = df[emotion_labels].apply(lambda x : [emotion for emotion in emotion_labels if x[emotion] == 1],axis = 1)

In [9]:
df = df[df['emotions'].map(len) > 0]

In [10]:
df = df[['text','emotions']]

In [11]:
print(df[df['emotions'].map(len) > 1])

                                                     text  \
13                                           That is odd.   
15      I appreciate it, that's good to know. I hope I...   
23      Now I'm wondering on what I've been missing ou...   
29      So happy for [NAME]. So sad he's not here. Ima...   
32      Dark and funny, but not really nice guy. He ha...   
...                                                   ...   
211187  I just called the Capitol Police. They are not...   
211198  Not true. backtested analysis shows that 85% s...   
211202                        Yes!!!!!! She’s a hot mess!   
211212    What a great photo and you two look so happy. 😍   
211217     You got banned for participating in a brigade.   

                                  emotions  
13               [disappointment, disgust]  
15                 [admiration, gratitude]  
23                  [curiosity, gratitude]  
29                 [disgust, joy, sadness]  
32               [disappointment, disgust]

In [12]:
df.to_csv("goemotions_filtered.csv", index=False)

# EDA ( Exploratory Data Analysis)

In [13]:
# plt.rcParams["figure.figsize"] = (12,6)
# emotion_counts = df[emotion_labels].sum().sort_values(ascending=False)

KeyError: "None of [Index(['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',\n       'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',\n       'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',\n       'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',\n       'relief', 'remorse', 'sadness', 'surprise'],\n      dtype='object')] are in the [columns]"

In [14]:
# print(emotion_counts)

NameError: name 'emotion_counts' is not defined

In [ ]:
# --- 1. Bar chart of emotion frequency ---



# Prepare data

In [13]:
for label in emotion_labels:
    df[label] = df['emotions'].apply(lambda x: int(label in x))

In [14]:
emotions = df.columns.tolist()[2:]
print(emotions)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']


In [17]:
# Define your emotion labels
from transformers import AutoTokenizer
import torch

emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
    'remorse', 'sadness', 'surprise'
]
num_labels = len(emotion_labels)

# Tokenize texts
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
encodings = tokenizer(
    list(df['text']),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

# Labels to tensor
labels = torch.tensor(df[emotion_labels].values).float()

In [18]:
print(type(encodings))
print(encodings.items())
print(labels)

<class 'transformers.tokenization_utils_base.BatchEncoding'>
ItemsView({'input_ids': tensor([[  101,  2008,  2208,  ...,     0,     0,     0],
        [  101,  2158,  1045,  ...,     0,     0,     0],
        [  101,  2157,  1029,  ...,     0,     0,     0],
        ...,
        [  101,  2092,  2043,  ...,     0,     0,     0],
        [  101,  2008,  3504,  ...,     0,     0,     0],
        [  101,  1996, 17473,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})
tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])


# Labels

In [20]:
from torch.utils.data import Dataset

class GoEmotionsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

dataset = GoEmotionsDataset(encodings, labels)

# Create PyTorch Dataset

In [21]:
class GoEmotionsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

dataset = GoEmotionsDataset(encodings, labels)

# Train / Test Split

In [22]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [23]:
print(type(train_dataset))

<class 'torch.utils.data.dataset.Subset'>


# Now we have to define the model

In [28]:
import torch.nn as nn
from transformers import AutoModel

class BERTMultiLabelClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BERTMultiLabelClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("distilbert-base-uncased")
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = output.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)
        return logits

# Train the Model

In [34]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BERTMultiLabelClassifier(num_labels)
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [35]:
epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}")

Epoch 1/3 - Loss: 0.1328
Epoch 2/3 - Loss: 0.1165
Epoch 3/3 - Loss: 0.1100


In [37]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).int()

        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

# Convert lists of tensors to numpy arrays
all_preds = torch.cat(all_preds, dim=0).numpy()
all_labels = torch.cat(all_labels, dim=0).numpy()


In [38]:
from sklearn.metrics import classification_report

print(classification_report(all_labels, all_preds, target_names=emotion_labels, zero_division=0))

                precision    recall  f1-score   support

    admiration       0.65      0.60      0.62      3434
     amusement       0.63      0.69      0.66      1834
         anger       0.53      0.36      0.43      1620
     annoyance       0.65      0.04      0.07      2710
      approval       0.55      0.19      0.29      3488
        caring       0.49      0.28      0.36      1218
     confusion       0.59      0.18      0.28      1471
     curiosity       0.59      0.46      0.51      2035
        desire       0.50      0.32      0.39       776
disappointment       0.49      0.07      0.13      1626
   disapproval       0.58      0.16      0.25      2328
       disgust       0.58      0.12      0.20      1071
 embarrassment       0.55      0.18      0.27       476
    excitement       0.59      0.14      0.22      1163
          fear       0.60      0.47      0.52       636
     gratitude       0.88      0.78      0.83      2332
         grief       0.27      0.04      0.08  

In [39]:
i = 5  # Any index
text = df['text'].iloc[i]
true_emotions = [emotion_labels[j] for j in range(len(emotion_labels)) if all_labels[i][j] == 1]
pred_emotions = [emotion_labels[j] for j in range(len(emotion_labels)) if all_preds[i][j] == 1]

print(f"Text: {text}")
print(f"True Emotions: {true_emotions}")
print(f"Predicted Emotions: {pred_emotions}")

Text: that's adorable asf
True Emotions: ['surprise']
Predicted Emotions: []


In [40]:
def predict_emotions(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs)
        probs = torch.sigmoid(logits)
        predicted = (probs > 0.5).int().squeeze()

    return [emotion_labels[i] for i, val in enumerate(predicted) if val == 1]

# Example:
predict_emotions("I feel excited but also a bit nervous about tomorrow.")

['excitement', 'nervousness']

In [41]:
import torch

torch.save(model.state_dict(), "bert_emotion_model.pt")

In [ ]:
model = BERTMultiLabelClassifier(num_labels)
model.load_state_dict(torch.load("bert_emotion_model.pt"))
model.to(device)
model.eval()